In [17]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import os

importlib.reload(repository)
importlib.reload(model)
importlib.reload(service)

<module 'backend.service_layer.service' from 'c:\\Users\\Site Oracle\\reduceComplexity\\backend\\service_layer\\service.py'>

In [18]:
from backend.domain.model import Company, Filing

In [19]:
uow_instance = uow.UnitOfWork()
company = service.get_company_by_ticker('GES', uow_instance)


In [20]:
company.filings = company.get_filings_by_type('10-K')

In [21]:
company.filings

In [22]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


Loaded data for filing date 2025-04-11, form 10-K


In [23]:
company.filings[0].income_statement.table

Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')


date_range,2022-01-30:2023-01-28,2023-01-29:2024-02-03,2024-02-04:2025-02-01
metric,,,
AssetImpairmentCharges,9.522000e+06,6.893500e+06,6.612000e+06
CostOfGoodsAndServicesSold,1.538603e+09,1.553950e+09,1.694283e+09
EarningsPerShareBasic,2.620000e+00,3.670000e+00,1.150000e+00
EarningsPerShareDiluted,2.180000e+00,3.090000e+00,7.700000e-01
GainLossOnTerminationOfLease,2.283500e+06,1.681000e+06,7.090000e+05
GainsLossesOnExtinguishmentOfDebt,0.000000e+00,-1.235100e+07,-1.952000e+06
GrossProfit,1.148747e+09,1.222580e+09,1.300990e+09
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,1.980460e+08,2.361350e+08,8.047300e+07
IncomeLossFromEquityMethodInvestments,0.000000e+00,0.000000e+00,-4.090000e+05


In [24]:
def get_skip_amount(last_filing: model.Filing, form_type: str):
    if last_filing.data and last_filing.income_statement:
        print(f"Years covered in last filing ({last_filing.filing_date}):")
        print(last_filing.income_statement.table.columns)
    else:
        last_filing.data = uow_instance.sec_filings.get_filing_data(
                                                last_filing.cik,
                                                last_filing.accession_number,
                                                last_filing.primary_document
                                            )
        print(last_filing.income_statement.table.columns)
    year_interval = len(last_filing.income_statement.table.columns)
    return (year_interval - 1) * 3 if form_type == '10-Q' else (year_interval - 1) * 1


In [25]:
def get_most_recent_filing(filings_list: list[model.Filing], form_type: str):
    if form_type == '10-K':
        return filings_list[0]
    elif form_type == '10-Q':
        return filings_list[3]


In [26]:
def select_filings_with_processing_pattern(
    filings_list: List[Filing], 
    form_type: str
) -> List[Filing]:
    

    if form_type == '10-K':
        process_count = 1
    elif form_type == '10-Q':
        process_count = 3
    
    skip_count = get_skip_amount(filings_list[0], form_type)
    
    
    if process_count <= 0:
        return []

    selected = []
    idx = 0
    total_filings = len(filings_list)

    while idx < total_filings:
        # Process N filings
        end_process_idx = min(idx + process_count, total_filings)
        selected.extend(filings_list[idx:end_process_idx])
        idx = end_process_idx

        # If we've processed all filings, break
        if idx >= total_filings:
            break

        # Skip M filings
        idx += skip_count

    # Add the last filing if it's not already in the list
    if form_type == '10-K':
        if filings_list[-1] not in selected:
            selected.append(filings_list[-1])
    elif form_type == '10-Q':
        for filing in filings_list[-3:]:
            if filing not in selected:
                selected.append(filing)
    
    return selected

In [27]:
selected_filings = select_filings_with_processing_pattern(company.filings, '10-K')

Years covered in last filing (2025-04-11):
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')
Index(['2022-01-30:2023-01-28', '2023-01-29:2024-02-03',
       '2024-02-04:2025-02-01'],
      dtype='object', name='date_range')


In [28]:
for filing in selected_filings:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2025-04-11, form 10-K
Loaded data for filing date 2022-03-24, form 10-K
Loaded data for filing date 2019-03-29, form 10-K
Loaded data for filing date 2016-03-25, form 10-K
Loaded data for filing date 2013-04-01, form 10-K
Loaded data for filing date 2010-03-31, form 10-K
Loaded data for filing date 2007-03-01, form 10-K


In [29]:
for filing in company.filings:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2025-04-11, form 10-K
Loaded data for filing date 2024-04-01, form 10-K
Loaded data for filing date 2023-03-24, form 10-K
Loaded data for filing date 2022-03-24, form 10-K
Loaded data for filing date 2021-04-09, form 10-K
Loaded data for filing date 2020-04-01, form 10-K
Loaded data for filing date 2019-03-29, form 10-K
Loaded data for filing date 2018-03-29, form 10-K
Loaded data for filing date 2017-03-27, form 10-K
Loaded data for filing date 2016-03-25, form 10-K
Loaded data for filing date 2015-03-27, form 10-K
Loaded data for filing date 2014-03-28, form 10-K
Loaded data for filing date 2013-04-01, form 10-K
Loaded data for filing date 2012-03-23, form 10-K
Loaded data for filing date 2011-03-28, form 10-K
Loaded data for filing date 2010-03-31, form 10-K
Loaded data for filing date 2009-03-31, form 10-K
Loaded data for filing date 2008-04-01, form 10-K
Loaded data for filing date 2007-03-01, form 10-K
